<a href="https://colab.research.google.com/github/lab30041954/ML_Course/blob/main/Assignments/Assignment%202%20-%20The%20prof%20solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 - The prof solution

### Task

Change the features matrix by: (a) dropping the three `cap_` features and (b) **binarizing** all the `word_` features, transforming every column into a dummy for the occurrence of the corresponding word, taking value 1 if the word occurs in the message and 0 otherwise. Based on this new features matrix, train two new spam filters, one based on a logistic regression model and the other one based on a decision tree model, using the binarized data set. Evaluate these new filters and compare them to those obtained before.

### Preparation

We start by creating the features matrix `X` and the target vector `y`, as usual in supervised learning. The following code chunk has already been used in class.

In [1]:
import pandas as pd
path = 'https://raw.githubusercontent.com/lab30041954/Data/main/'
df = pd.read_csv(path + 'spam.csv')
y = df['spam']
X = df.drop(columns='spam')

### Evaluation

We create a function for the evaluation of our models on the raw data set, packing the calculations already presented in class. We don't the accuracy, because these data has a spam rate which is not the real one. So we perform separate evaluations on the spam and the legal email.

In [2]:
from sklearn.metrics import confusion_matrix
def eval1(clf):
        y_pred = clf.predict(X)
        conf = confusion_matrix(y, y_pred)
        tp = conf[1, 1]/sum(conf[1, :])
        fp = conf[0, 1]/sum(conf[0, :])
        return round(tp, 3), round(fp, 3)


### Logistic regression classifier (1)

Firs, we train a logistic regression classifier. Since the data are not normalized, we need to increase the number of iterations to achieve convergence.

In [3]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(max_iter=3000)
clf1.fit(X, y)
eval1(clf1)

(np.float64(0.881), np.float64(0.053))

The true positive rate is quite good (88.1%), while we keep the false negative rate, which is our primary focus, at a moderate level (5.3%).

### Decision tree classifier (1)

Next, we train the last of the decision tree classifiers presented in class. The performance is a bit weaker than that of the logistic regression model.

In [4]:
from sklearn.tree import DecisionTreeClassifier
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=5)
clf2.fit(X, y)
eval1(clf2)

(np.float64(0.841), np.float64(0.057))

We take note of the importance ranking, for reference. One of the features that we are going to remove (`cap_ave`) is in the fourth place, which indicates that the removal can affect the performance of the decision tree models.

In [5]:
imp2 = pd.Series(clf2.feature_importances_, index=X.columns)
imp2[imp2 > 0].sort_values(ascending=False).round(3)

word_remove     0.359
word_free       0.204
word_hp         0.130
cap_ave         0.113
word_george     0.059
word_money      0.045
word_edu        0.034
cap_long        0.022
word_receive    0.013
word_your       0.007
word_1999       0.004
word_650        0.004
word_mail       0.003
word_hpl        0.002
dtype: float64

### Binary data set

We create the new data set as follows: (a) by picking `X.iloc[:, :-3]` we leave aside the last three columns, (b) with the expression `X.iloc[:, :-3] > 0` we replace the original values by Boolean dummies (`True` when the original value is positive and `False` otherwise), and (c) with `.astype('int')` we convert the Boolean values to ones and zeros. This last conversion is not really needed, and it is included to follow literally the instructions given. We call `BX` the new binary data set.

In [6]:
BX = (X.iloc[:, :-3] > 0).astype('int')
BX.head()

,word_make,word_address,word_all,word_3d,word_our,word_over,word_remove,word_internet,word_order,word_mail,...,word_pm,word_direct,word_cs,word_meeting,word_original,word_project,word_re,word_edu,word_table,word_conference
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,1,1,1,1,1,1,...,0,1,0,0,1,0,1,1,0,0
3,0,0,0,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


Next, we create a function for the evaluation of our models on the binary data set.

In [7]:
def eval2(clf):
        y_pred = clf.predict(BX)
        conf = confusion_matrix(y, y_pred)
        tp = conf[1, 1]/sum(conf[1, :])
        fp = conf[0, 1]/sum(conf[0, :])
        return round(tp, 3), round(fp, 3)

### Logistic regression classifier (2)

We train a logistic regression classifier on the binary data set. We don't need extra iterations now, since all the features are dummies. The performance is about the same as on the original data set, in spite of the information "lost".

In [8]:
clf3 = LogisticRegression()
clf3.fit(BX, y)
eval2(clf3)

(np.float64(0.881), np.float64(0.048))

### Decision tree classifier (2)

On the other hand, the performance of the decision tree classifier is a bit weaker on the binary data set. This could be attributed to the removal of one of the relevant features.

In [9]:
clf4 = DecisionTreeClassifier(criterion='entropy', max_depth=5)
clf4.fit(BX, y)
eval2(clf4)

(np.float64(0.821), np.float64(0.052))

The new importance ranking does not change in the three top places, and the rest is adapted to new setting.

In [10]:
imp4 = pd.Series(clf4.feature_importances_, index=BX.columns)
imp4[imp4 > 0].sort_values(ascending=False).round(3)

word_remove        0.382
word_free          0.206
word_hp            0.116
word_money         0.102
word_internet      0.051
word_george        0.051
word_edu           0.047
word_credit        0.013
word_1999          0.007
word_your          0.006
word_650           0.005
word_business      0.003
word_labs          0.003
word_conference    0.002
word_meeting       0.002
word_order         0.002
word_000           0.001
dtype: float64